In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import ubergauss.tools as ut
from yoda import graphs as ygraphs
import subprocess
import seaborn as sns
from umap import UMAP
from yoda import draw
import ubergauss.optimization as uo 
import pandas as pd
import smallgraph as sg
import networkx as nx
from matplotlib import pyplot as plt
import lmz
import eden.graph as eg
from yoda.graphs import ali2graph
import numpy as np
import yoda.ml.simpleMl as sml
import eden.display as ed
from yoda.alignments import load_rfam, filter_by_seqcount
import ubergauss.tools as ut
from yoda import graphs as ygraphs
from sklearn.preprocessing import normalize


import matplotlib as mpl
mpl.rc("figure", dpi=300)

In [ ]:
import cmcomp
data = {}

# CMCOMPARE

In [ ]:
a,l = load_rfam(full= False, add_cov= False)
names = [aa.gf["AC"][3:] for aa in a]

# THIS IS HOW WE WOULD RERUN THE CALCULATION
# [cmcomp.dumpcm(name) for name in names]
# IMPORTANTRESULT = cmcomp.run_cmcompare_pairwise()
# IMPORTANTRESULT.to_csv('cmcompare_full_run')

# LOAD CMCOMP DATA 
cmcompare_data = pd.read_csv('cmcompare_full_run_2024_06_27')
cmcompare_data  = cmcompare_data.fillna(500) 
cmcompare_data['score1'] = cmcompare_data[['score', 'score2']].min(axis=1)
cmcompare_data['score3'] = cmcompare_data[['score', 'score2']].max(axis=1)





In [ ]:

# the df has similarities.. 
cmcompare_dist = cmcomp.to_dist(cmcomp.pivot_numpy(cmcompare_data))
data['CMCompare'] = cmcompare_dist



# YODA 

In [ ]:

import pandas as pd
from lmz import Range
from sklearn.metrics.pairwise import euclidean_distances
def atograph(ali):
    graph = ali2graph.writecons(ali)
    graph = ali2graph.dillute( graph)
    # graph = ali2graph.conscut( graph,  consThresh= .95, replacelabel = False )
    graphs = ali2graph.set_weight( graph  )
    # graphs = ali2graph.nearstem( graph, boost_range = 4, boost_thresh = .5, boost_weight = 1)
    graphs = ali2graph.donest( graph)
    return graph

graphs = ut.xxmap(atograph, a)
matrix = ygraphs.vectorize_graphs(graphs, normalization=True,
                      min_r = 2,min_d=1,
                      inner_normalization=True)


def mkdf(mat): 
    sim = mat @ mat.T
    # sim = -euclidean_distances(mat)
    data = [{'cm1':a, 'cm2':b , 'score1': sim[a,b]} for a in Range(l) for b in Range(l)]
    return pd.DataFrame(data)
    
#yoda_data = mkdf(matrix)
#yoda_dist = cmcomp.to_dist(cmcomp.pivot_numpy(yoda_data))

from sklearn.metrics import euclidean_distances

def mkdist(mat):
    return  euclidean_distances(mat)
    mat = (mat@mat.T).toarray()
    mat+=abs(mat.min())
    np.fill_diagonal(mat,0)
    return mat



In [ ]:
ourmethod = 'KRAID'

data[ourmethod] = mkdist(matrix)

In [ ]:

a,l = load_rfam(full= False, add_cov= False)
graphs = lmz.Map(ali2graph.writecons, a)
eden_sim = ygraphs.vectorize_graphs(graphs, normalization=True,
                      min_r = 2,min_d=1,
                      inner_normalization=True)

data['NSPDK'] = mkdist(eden_sim)


# INFERNAL

In [ ]:
import inferencemodel as inf

def infernal_sim_to_dist(SIM_INF):
    infernal_dist = cmcomp.to_dist(SIM_INF)
    indices = np.where(infernal_dist == SIM_INF.max())
    noise = np.random.normal(0, 1, size=len(indices[0]))
    infernal_dist[indices] += noise
    return infernal_dist

SIM_INF_global = inf.readCmscanAndMakeTable((a,l), path = 'inftools/infernal_global.tbl')
SIM_INF = inf.readCmscanAndMakeTable((a,l), path = 'inftools/infernal.tbl')

data['Infernal']=infernal_sim_to_dist(SIM_INF)
data['Infernal_global']=infernal_sim_to_dist(SIM_INF_global)




In [ ]:
if False:
    from kiez import Kiez
    def normcsls(di):
        k_inst = Kiez(algorithm='SklearnNN', hubness='csls',  n_candidates = di.shape[0]-1,  
                  algorithm_kwargs= {'metric' : 'precomputed'})
        k_inst.fit(di)
        dist2, indices2 = k_inst.kneighbors(di.shape[0]-1)
        
        n = dist2.shape[0]
        complete_distance_matrix = np.zeros((n, n))
        # Fill the known distances
        for i, neighbors in enumerate(indices2):
            complete_distance_matrix[i, neighbors] = dist2[i]
        return complete_distance_matrix

# clan discovery

In [ ]:
randomarray = np.random.rand(395,395)
np.fill_diagonal(randomarray, 0)
data['random']= randomarray

In [ ]:
data

In [ ]:
# k clans plot 
# we also need to collect the results for the other normed methods -> 3 x 2 lines are in this plot 
from yoda.ml import nearneigh
r = []
ylabel = 'Label Hit Rate'
for k,dist in data.items():
    for i,val in enumerate( cmcomp.k_clan_discovery(dist,l,50)[1:]):
        r+=[{'Distances':'unmodified','Method':k,'neighbors':i+1,ylabel:val}]
    
    # dist_norm = normalize(dist, axis=0)
    dist_norm = nearneigh.normalize_csls(dist)
    dist_norm += np.abs(dist_norm.min())

    for i,val in enumerate( cmcomp.k_clan_discovery(dist_norm,l,50)[1:]):
        r+=[{'Distances':'normalized','Method':k,'neighbors':i+1,ylabel:val}]

df = pd.DataFrame(r)

#sns.reset_defaults()
# FULL PLOT 
sns.set_theme()
sns.set_context("talk")
ax= sns.lineplot(df, x= 'neighbors', y= ylabel,hue='Method', style='Distances') 
sns.move_legend( ax,"center left", bbox_to_anchor=(1, 0.5))
plt.title('Hit Rate of RNA alignments\nwith respect to their clan')
plt.show()
plt.close()

#sns.reset_defaults()
# REDUCED PLOT 
df = df[df.Distances == 'normalized']
df = df[df.Method != 'Infernal_global']
sns.set_theme()
sns.set_context("talk")
ax= sns.lineplot(df, x= 'neighbors', y= ylabel,hue='Method') 
sns.move_legend( ax,"center left", bbox_to_anchor=(1, 0.5))
plt.title('Hit Rate of RNA alignments\nwith respect to their clan')
plt.show()
plt.close()

In [ ]:
# precision recall, there are also 3 methods and a hue that will be the normalisation, 
# we make a dataframe oO 
# we need to report mAP and AUC seperately i think

In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_curve, auc
import matplotlib.pyplot as plt

def get_pairwise_distances(dist_matrix, labels):
    pairwise_distances = []
    pairwise_labels = []
    n = dist_matrix.shape[0]

    for i in range(n):
        for j in range(i+1, n):
            pairwise_distances.append(dist_matrix[i][j])
            pairwise_labels.append(labels[i] == labels[j]) 

    return np.array(pairwise_distances), np.array(pairwise_labels)


def prec_recall_plot(distances, labels):
    distances, true_labels = get_pairwise_distances(distances, labels)
    precision, recall, thresholds = precision_recall_curve(true_labels, -distances) 
    return precision, recall
    # Plot Precision-Recall curve




r = []
for k,dist in data.items():
    
    for p,re in zip(*prec_recall_plot(dist,l)):
        r+=[{'Distances':'Raw','Method':k,'precision':p,'recall':re}]
    
    # dist_norm = normalize(dist, axis=0)
    # dist_norm = normcsls(dist)
    dist_norm = nearneigh.normalize_csls(dist)

    for p,re in zip(*prec_recall_plot(dist_norm,l)):
        r+=[{'Distances':'Normalized','Method':k,'precision':p,'recall':re}]
        
df = pd.DataFrame(r)


#sns.reset_defaults()
sns.set_theme()
sns.set_context("talk")
ax= sns.lineplot(df, y= 'precision', x= 'recall', hue='Method', style='Distances') 
sns.move_legend( ax,"center left", bbox_to_anchor=(1, 0.5))
plt.show()
plt.close()

In [ ]:
d = []
for k,dist in data.items(): 
    #dist_norm = normalize(dist, axis=0)
    dist_norm = nearneigh.normalize_csls(dist)

    p,r= prec_recall_plot(dist,l)
    AUC = auc(r,p) 
    print(f"{k} raw AUC:{AUC}")
    mAP = cmcomp.mAP(dist,l)
    print(f"{k} raw mAP:{mAP}")
    d.append({'method':k, 'scoretype':'AUC', 'normalized':'no', 'score':AUC})
    d.append({'method':k, 'scoretype':'mAP', 'normalized':'no', 'score':mAP})
    
    p,r= prec_recall_plot(dist_norm,l)
    AUC = auc(r,p) 
    print(f"{k} normalized AUC {AUC}")
    mAP = cmcomp.mAP(dist_norm,l)
    print(f"{k} normalized mAP {mAP}")
    d.append({'method':k, 'scoretype':'AUC', 'normalized':'yes', 'score':AUC})
    d.append({'method':k, 'scoretype':'mAP', 'normalized':'yes', 'score':mAP})



In [ ]:
df = pd.DataFrame(d)

In [ ]:

df2 = df.pivot_table(index='method', columns=['scoretype', 'normalized'], values='score', fill_value=0)


In [ ]:


df2.to_latex(index=False,  header=True)

In [ ]:
df2